# **Ntuples with showers - plots**

Code to generate the `.root` histograms used here is available in [github.com/DanielEstrada971102/DTPatternRecognition](https://github.com/DanielEstrada971102/DTPatternRecognition/tree/destrada-dev-Event).

---------------------------------------------
# **Setup notebook**

In [1]:
import sys
import os
import re
from copy import deepcopy
import numpy as np

In [2]:
if 'ROOT' in sys.modules:
    import ROOT as r
    %jsroot on
else:
    !wget https://github.com/MohamedElashri/ROOT/releases/download/ubuntu/root_v6.28.04_Ubuntu_20.04.zip -q --show-progress
    !unzip /content/root_v6.28.04_Ubuntu_20.04.zip >> log.txt
    !apt-get install -y git dpkg-dev cmake g++ gcc binutils libx11-dev libxpm-dev libxft-dev libxext-dev tar gfortran subversion >> log.txt

    # The following is needed because colab upgraded the openssl library
    !wget http://archive.ubuntu.com/ubuntu/pool/main/o/openssl/libssl1.1_1.1.1f-1ubuntu2_amd64.deb -q --show-progress>> log.txt
    !sudo dpkg -i libssl1.1_1.1.1f-1ubuntu2_amd64.deb >> log.txt

    import sys
    sys.path.append("/content/root_build/")
    sys.path.append("/content/root_build/bin/")
    sys.path.append("/content/root_build/include/")
    sys.path.append("/content/root_build/lib/")
    import ctypes
    ctypes.cdll.LoadLibrary('/content/root_build/lib//libCore.so')
    ctypes.cdll.LoadLibrary('/content/root_build/lib//libThread.so')
    ctypes.cdll.LoadLibrary('/content/root_build/lib//libTreePlayer.so')

    import ROOT as r
    %jsroot on

root_v6.28.04_Ubunt 100%[===================>] 227.84M  81.2MB/s    in 2.8s    
libssl1.1_1.1.1f-1u 100%[===================>]   1.26M  7.78MB/s    in 0.2s    
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 78.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
/sbin/ldconfig.real: /usr/local/lib/libhwloc.so.15 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libtbbmalloc.so.2 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libtbbbind_2_5.so.3 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libtbbmalloc_proxy.so.2 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libur_adapter_opencl.so.0 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libtcm.s

-----------------------------------------------------------------------
# **Get Data**

In [3]:
#@markdown * `/results` is available in `lustrefs/hdd_pool_dir/L1T/Filter/ThresholdScan_Zprime_DY/results/` or in this [cernbox_link](https://cernbox.cern.ch/s/riFcDzF2D4DQiSo)
#@markdown * `histograms_AM_withShowers_MAIN.root` is any of the files in  `/results`, or just use this [one](https://cernbox.cern.ch/s/nz8vurg0PJh48IZ)

#@markdown file names are saved to be used along the notebook in:
#@markdown - **files** : `dict` which keys are the different thresholds -> `(12, 52, 4)`
#@markdown - **baseFile**: `histograms_AM_withShowers_MAIN.root`

if not os.path.isdir("results"):
    print("Downloading files...")
    !wget https://cernbox.cern.ch/remote.php/dav/public-files/ZQcnCCiMvGc94Nf/data_showers.tar #-q --show-progress
    !tar -xvf data_showers.tar
    print("Done!")
!wget https://cernbox.cern.ch/remote.php/dav/public-files/Dp0m16g7zI086yK/histograms_AM_withShowers_thr20.root #-q --show-progress
inputpath = "./results"
# input file names, accessible with thrX
files = {re.findall(r'thr\d+', inputfile)[0]: os.path.join(inputpath, inputfile) for inputfile in os.listdir(inputpath)}
baseFile = os.path.join("./histograms_AM_withShowers_MAIN.root")
thr20File = os.path.join("./histograms_AM_withShowers_thr20.root")

--2024-11-15 13:43:33--  https://cernbox.cern.ch/remote.php/dav/public-files/ZQcnCCiMvGc94Nf/data_showers.tar
Resolving cernbox.cern.ch (cernbox.cern.ch)... 137.138.120.151, 128.142.53.35, 128.142.53.28, ...
Connecting to cernbox.cern.ch (cernbox.cern.ch)|137.138.120.151|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 919552 (898K) [application/x-tar]
Saving to: ‘data_showers.tar’

data_showers.tar    100%[===================>] 898.00K  1.35MB/s    in 0.7s    

2024-11-15 13:43:37 (1.35 MB/s) - ‘data_showers.tar’ saved [919552/919552]

results/
results/histograms_AM_withShowers_thr12.root
results/histograms_AM_withShowers_thr16.root
results/histograms_AM_withShowers_thr20.root
results/histograms_AM_withShowers_thr24.root
results/histograms_AM_withShowers_thr28.root
results/histograms_AM_withShowers_thr32.root
results/histograms_AM_withShowers_thr36.root
results/histograms_AM_withShowers_thr40.root
results/histograms_AM_withShowers_thr44.root
results/histograms

--------------------------------------------------------------
# **gen muons plots**

In [4]:
#@title variables
f = r.TFile.Open(baseFile)
c =r.TCanvas("canvas1", "", 1000, 500)
c.Divide(2,2)
histos = []
legends = []
titles = {"pt": "p_{T}", "eta": "#eta", "maxDPhi": "max #Delta#phi", "Max_dEta":"max #Delta#eta"}

for i, (name, title) in enumerate(titles.items()):
    c.cd(i+1)

    histo1 = deepcopy(f.Get("LeadingMuon_"+name))
    histo1.SetFillColorAlpha(r.kRed, 0.5)
    histo1.SetTitle(title)
    histo1.GetXaxis().SetTitleSize(10)
    histo2 = deepcopy(f.Get("SubLeadingMuon_"+name))
    histo2.SetFillColorAlpha(r.kBlue, 0.5)

    x0leg, y0leg, x1leg, y1leg = (0.62, 0.37, 0.80, 0.55)
    legends.append(r.TLegend(x0leg, y0leg, x1leg, y1leg))
    legends[-1].SetBorderSize(0)
    legends[-1].SetFillColor(0)
    legends[-1].SetTextFont(42)
    legends[-1].SetTextSize(15)
    legends[-1].SetNColumns(1)
    legends[-1].AddEntry(histo1,"Leading #mu")
    legends[-1].AddEntry(histo2,"SubLeading #mu")

    histos.append([histo1, histo2])

    histo1.Draw("HIST")
    histo2.Draw("SAMEHIST")
    legends[-1].Draw("SAME")

c.Draw()
f.Close()

-----------------------------------------------------------------------
# **Digis plots**

In [5]:
#@title functions
def make_per_whst_2dhisto(h, histos, vlines_range = (0, 200)):
    # get histograms from histos and fill h
    for iWheel in range(1, 6):
        for iStation in range(1, 5):
            iBin = 4 * (iStation-1) + iWheel + (iStation != 1)*(iStation - 1)
            for iWire in range(1, histos[ iStation - 1 ].GetNbinsY() + 1):
                h.SetBinContent( iBin, iWire, histos[ iStation - 1 ].GetBinContent(iWheel, iWire))

    # set bin lebels
    labels = ["-2", "-1", "0", "+1", "+2"]*4
    for iBin in range(h.GetNbinsX()):
        h.GetXaxis().SetBinLabel(iBin + 1, labels[iBin])

    h.SetStats(0) # turn off stats box
    h.Draw("COLZ")

    # customize
    notes = [
        ("MB1", .2, .85, 0.05),
        ("MB2", .4, .85, 0.05),
        ("MB3", .6, .85, 0.05),
        ("MB4", .8, .85, 0.05),
    ]
    lines = [
        (5, vlines_range[0], 5, vlines_range[1]),
        (10, vlines_range[0], 10, vlines_range[1]),
        (15, vlines_range[0], 15, vlines_range[1]),
    ]

    _noDelete = []
    # Now add texts and lines
    for note in notes:
        text = note[0]
        x, y = note[1], note[2]
        textSize = note[3]
        latex = r.TLatex()
        latex.SetTextSize(textSize)
        latex.SetTextAlign(22)  # Centrado
        latex.SetTextFont(42)
        latex.DrawLatexNDC(x, y, text)
        _noDelete.append(latex)  # Para evitar que ROOT lo borre

    for line in lines:
            xpos0, ypos0, xpos1, ypos1 = line
            texline = deepcopy(r.TLine(xpos0, ypos0, xpos1, ypos1))
            texline.SetLineWidth(1)
            texline.Draw("same")
            _noDelete.append(texline) # So it does not get deleted by ROOT

    return _noDelete # return to ovoid the plot disappear

def draw_projection(h, bin=("MB1", -2), color=r.kBlue):
    # get a TH1 projection
    binx = (bin[1] + 3) + 5 * (int(re.findall("\d+", bin[0])[0]) - 1)
    histo_p = deepcopy(h.ProjectionY("", binx, binx))
    histo_p.SetTitle(f"Projection of {bin[0]} - wheel {bin[1]}; # Wire;")
    histo_p.GetXaxis().SetTitleFont(42)
    histo_p.GetXaxis().SetTitleSize(0.03)
    histo_p.GetXaxis().SetLabelFont(42)
    histo_p.GetXaxis().SetLabelSize(0.025)
    histo_p.GetYaxis().SetLabelFont(42)
    histo_p.GetYaxis().SetLabelSize(0.025)
    histo_p.GetXaxis().SetTitleOffset(1.5)

    x0, x1 = h.GetXaxis().GetBinLowEdge(binx), h.GetXaxis().GetBinUpEdge(binx)
    y0, y1 = histo_p.GetBinLowEdge(1), histo_p.GetBinCenter(histo_p.GetNbinsX())

    box = r.TBox(x0, y0, x1, y1)
    box.SetFillColorAlpha(color, 0.3)
    return histo_p, box

In [6]:
#@title Simple digi ocupancy
#@markdown 2D histogram with the information of digis' wire number per Wheel and Station (right: for events with gen muons that didn't "shower"; left: for events with gen muons that did "shower"). see [baseHistos.py#L378](https://github.com/DanielEstrada971102/DTPatternRecognition/blob/a3988f0875b60d1eca1ca19531b1c0e2cbe4658e/utils/baseHistos.py#L378)
station = "MB1" # @param ["MB1", "MB2", "MB3", "MB4"]
wheel =  "0" # @param [-2, -1, 0, 1, 2]
wheel = int(wheel)

f = r.TFile.Open(thr20File)
c2 = r.TCanvas("canvas2", "", 1000, 800)
h2d_1 = r.TH2I("h2d_1", "Ocupancy of the digis' wires for events with not showered genmuons",20, 0, 20, 200, 0, 200)
histos1 = [ deepcopy( f.Get( f"digi_w_gmtns_MB{ist}")) for ist in range(1, 5) ]
h2d_2 = r.TH2I("h2d_2", "Ocupancy of the digis' wires for events with showered genmuons",20, 0, 20, 200, 0, 200)
histos2 = [ deepcopy( f.Get( f"digi_w_gmts_MB{ist}")) for ist in range(1, 5) ]

c2.Divide(2, 2)

c2.cd(1)
_nd1 = make_per_whst_2dhisto(h2d_1, histos1)
pry_1, box_1 = draw_projection(h2d_1, (station, wheel), color=r.kRed)
box_1.Draw("same")

c2.cd(2)
_nd2 = make_per_whst_2dhisto(h2d_2, histos2)
pry_2, box_2 = draw_projection(h2d_2, (station, wheel), color=r.kGreen)
box_2.Draw("same")

c2.cd(3)
r.gPad.SetFrameLineColor(r.kRed)
r.gPad.SetFrameLineWidth(4)
pry_1.Draw()

c2.cd(4)
r.gPad.SetFrameLineColor(r.kGreen)
r.gPad.SetFrameLineWidth(4)
pry_2.Draw()

# Normalize color bar
min_value = min(h2d_1.GetMinimum(), h2d_2.GetMinimum())
max_value = max(h2d_1.GetMaximum(), h2d_2.GetMaximum())

h2d_1.SetMinimum(min_value)
h2d_1.SetMaximum(max_value)
h2d_2.SetMinimum(min_value)
h2d_2.SetMaximum(max_value)

pry_1.SetMinimum(min_value)
pry_1.SetMaximum(max_value)
pry_2.SetMinimum(min_value)
pry_2.SetMaximum(max_value)

# change axis names
h2d_1.GetXaxis().SetTitle("Wheel")
h2d_1.GetYaxis().SetTitle("# Wire")
h2d_2.GetXaxis().SetTitle("Wheel")
h2d_2.GetYaxis().SetTitle("# Wire")

c2.Draw()

In [7]:
#@title Distributions of $(w_i -\bar{w})$
#@markdown $w_i$ is the wire number of a digi. (right: for events with gen muons that didn't "shower"; left: for events with gen muons that did "shower"). see [baseHistos.py#L378](https://github.com/DanielEstrada971102/DTPatternRecognition/blob/a3988f0875b60d1eca1ca19531b1c0e2cbe4658e/utils/baseHistos.py#L378)
station = "MB1" # @param ["MB1", "MB2", "MB3", "MB4"]
wheel =  "-2" # @param [-2, -1, 0, 1, 2]
wheel = int(wheel)

f = r.TFile.Open(thr20File)
c3 = r.TCanvas("canvas3", "", 1000, 800)
h2d_3 = r.TH2I("h2d_3", "Distributions of (w_{i} - #bar{w}) for digis of events with not showered genmuons",20, 0, 20, 200, -100, 100)
histos3 = [ deepcopy( f.Get( f"digi_wd_gmtns_MB{ist}")) for ist in range(1, 5) ]
h2d_4 = r.TH2I("h2d_4", "Distributions of (w_{i} - #bar{w}) for digis of events with showered genmuons",20, 0, 20, 200, -100, 100)
histos4 = [ deepcopy( f.Get( f"digi_wd_gmts_MB{ist}")) for ist in range(1, 5) ]

c3.Divide(2, 2)

c3.cd(1)
_nd3 = make_per_whst_2dhisto(h2d_3, histos3, vlines_range=(-100, 100))
pry_3, box_3 = draw_projection(h2d_3, (station, wheel), color=r.kRed)
box_3.Draw("same")

c3.cd(2)
_nd4 = make_per_whst_2dhisto(h2d_4, histos4, vlines_range=(-100, 100))
pry_4, box_4 = draw_projection(h2d_4, (station, wheel), color=r.kGreen)
box_4.Draw("same")

c3.cd(3)
r.gPad.SetFrameLineColor(r.kRed)
r.gPad.SetFrameLineWidth(4)
pry_3.Draw()

c3.cd(4)
r.gPad.SetFrameLineColor(r.kGreen)
r.gPad.SetFrameLineWidth(4)
pry_4.Draw()


# Normalize color bar
min_value = min(h2d_3.GetMinimum(), h2d_4.GetMinimum())
max_value = max(h2d_3.GetMaximum(), h2d_4.GetMaximum())

h2d_3.SetMinimum(min_value)
h2d_3.SetMaximum(max_value)
h2d_4.SetMinimum(min_value)
h2d_4.SetMaximum(max_value)

pry_3.SetMinimum(min_value)
pry_3.SetMaximum(max_value)
pry_4.SetMinimum(min_value)
pry_4.SetMaximum(max_value)
# change axis names
h2d_3.GetXaxis().SetTitle("Wheel")
h2d_3.GetYaxis().SetTitle("(w_{i} - #bar{w})")
h2d_4.GetXaxis().SetTitle("Wheel")
h2d_4.GetYaxis().SetTitle("(w_{i} - #bar{w})")

c3.Draw()

In [8]:
#@title Distributions of $(w_{max} -w_{min})$
#@markdown $w_{max}$ is the maximun digi wire number, $w_{min}$ is the minimun. (right: for events with gen muons that didn't "shower"; left: for evetns with gen muons that did "shower"). see [baseHistos.py#L378](https://github.com/DanielEstrada971102/DTPatternRecognition/blob/a3988f0875b60d1eca1ca19531b1c0e2cbe4658e/utils/baseHistos.py#L378)
station = "MB1" # @param ["MB1", "MB2", "MB3", "MB4"]
wheel =  "2" # @param [-2, -1, 0, 1, 2]
wheel = int(wheel)

f = r.TFile.Open(thr20File)
c4 = r.TCanvas("canvas4", "", 1000, 800)
h2d_5 = r.TH2I("h2d_5", "Distributions of (w_{max} - w_{min}) for digis of events with not showered genmuons",20, 0, 20, 200, 0, 200)
histos5 = [ deepcopy( f.Get( f"digi_wl_gmtns_MB{ist}")) for ist in range(1, 5) ]
h2d_6 = r.TH2I("h2d_6", "Distributions of (w_{max} - w_{min}) for digis of events with showered genmuons",20, 0, 20, 200, 0, 200)
histos6 = [ deepcopy( f.Get( f"digi_wl_gmts_MB{ist}")) for ist in range(1, 5) ]

c4.Divide(2, 2)

c4.cd(1)
_nd5 = make_per_whst_2dhisto(h2d_5, histos5)
pry_5, box_5 = draw_projection(h2d_5, (station, wheel), color=r.kRed)
box_5.Draw("same")

c4.cd(2)
_nd6 = make_per_whst_2dhisto(h2d_6, histos6)
pry_6, box_6 = draw_projection(h2d_6, (station, wheel), color=r.kGreen)
box_6.Draw("same")

c4.cd(3)
r.gPad.SetFrameLineColor(r.kRed)
r.gPad.SetFrameLineWidth(4)
pry_5.GetXaxis().SetRangeUser(0,30)
pry_5.Draw()

c4.cd(4)
r.gPad.SetFrameLineColor(r.kGreen)
r.gPad.SetFrameLineWidth(4)
pry_6.GetXaxis().SetRangeUser(0,30)
pry_6.Draw()

# Normalize color bar
min_value = min(h2d_5.GetMinimum(), h2d_6.GetMinimum())
max_value = max(h2d_5.GetMaximum(), h2d_6.GetMaximum())

h2d_5.SetMinimum(min_value)
h2d_5.SetMaximum(max_value)
h2d_6.SetMinimum(min_value)
h2d_6.SetMaximum(max_value)

pry_5.SetMinimum(min_value)
pry_5.SetMaximum(max_value)
pry_6.SetMinimum(min_value)
pry_6.SetMaximum(max_value)

# change axis names
h2d_5.GetXaxis().SetTitle("Wheel")
h2d_5.GetYaxis().SetTitle("(w_{max} - w_{min})")
h2d_6.GetXaxis().SetTitle("Wheel")
h2d_6.GetYaxis().SetTitle("(w_{max} - w_{min})")

c4.Draw()

In [9]:
#@title functions2
def make_per_whstsl_3dhisto(h, histos, vlines_range = (0, 200)):
    # get histograms from histos and fill h
    for iStation in range(1, 5):  # 4 estaciones
        for iWheel in range(1, 6):  # 5 ruedas
            for iSL in range(1, 4):  # 3 subgrupos
                iBin = (iStation - 1) * 15 + (iWheel - 1) * 3 + iSL
                for iWire in range(1, histos[ iStation - 1 ].GetNbinsZ() + 1):
                    h.SetBinContent( iBin, iWire, histos[ iStation - 1 ].GetBinContent(iWheel, iSL, iWire))

    # labels = iter(["-2", "-1", "0", "+1", "+2"]*4)
    for iBin in range(1, h.GetNbinsX()+1):
        h.GetXaxis().SetBinLabel(iBin, str(iBin)) #next(labels) if iBin%3==0 else

    h.SetStats(0) # turn off stats box
    h.Draw("COLZ")

    # customize
    notes = [
        ("MB1", .2, .85, 0.05),
        ("MB2", .4, .85, 0.05),
        ("MB3", .6, .85, 0.05),
        ("MB4", .8, .85, 0.05),
    ]
    lines = [
        (15, vlines_range[0], 15, vlines_range[1]),
        (30, vlines_range[0], 30, vlines_range[1]),
        (45, vlines_range[0], 45, vlines_range[1]),
    ]

    _noDelete = []
    # Now add texts and lines
    for note in notes:
        text = note[0]
        x, y = note[1], note[2]
        textSize = note[3]
        latex = r.TLatex()
        latex.SetTextSize(textSize)
        latex.SetTextAlign(22)  # Centrado
        latex.SetTextFont(42)
        latex.DrawLatexNDC(x, y, text)
        _noDelete.append(latex)  # Para evitar que ROOT lo borre

    for line in lines:
            xpos0, ypos0, xpos1, ypos1 = line
            texline = deepcopy(r.TLine(xpos0, ypos0, xpos1, ypos1))
            texline.SetLineWidth(1)
            texline.Draw("same")
            _noDelete.append(texline) # So it does not get deleted by ROOT

    return _noDelete # return to ovoid the plot disappear

def draw_projection2(h, bin=("MB1", -2), color=r.kBlue):
    # get a TH1 projection
    mb = int(re.findall("\d+", bin[0])[0])
    wh = (bin[1] + 3)
    binx = (wh-1) * 3 + 15 * (mb-1) + 1

    histo_p_sl1 = deepcopy(h.ProjectionY("", binx, binx))
    histo_p_sl2 = deepcopy(h.ProjectionY("", binx+1, binx+1))
    histo_p_sl3 = deepcopy(h.ProjectionY("", binx+2, binx+2))
    histo_p_sl1.SetTitle(f"Projection of {bin[0]} - wheel {bin[1]}; # Wire;")
    histo_p_sl1.SetFillColorAlpha(8, 0.7)
    histo_p_sl2.SetFillColorAlpha(9, 0.7)
    histo_p_sl3.SetFillColorAlpha(11, 0.7)
    histo_p_sl1.GetXaxis().SetTitleFont(42)
    histo_p_sl1.GetXaxis().SetTitleSize(0.03)
    histo_p_sl1.GetXaxis().SetLabelFont(42)
    histo_p_sl1.GetXaxis().SetLabelSize(0.025)
    histo_p_sl1.GetYaxis().SetLabelFont(42)
    histo_p_sl1.GetYaxis().SetLabelSize(0.025)
    histo_p_sl1.GetXaxis().SetTitleOffset(1.5)

    x0leg, y0leg, x1leg, y1leg = (0.62, 0.37, 0.82, 0.57)
    l = r.TLegend(x0leg, y0leg, x1leg, y1leg)
    l.SetBorderSize(0)
    l.SetFillColor(0)
    l.SetTextFont(42)
    l.SetTextSize(30)
    l.SetNColumns(1)
    l.AddEntry(histo_p_sl1,"SL1")
    l.AddEntry(histo_p_sl2,"SL2")
    l.AddEntry(histo_p_sl3,"SL3")

    x0, x1 = h.GetXaxis().GetBinLowEdge(binx), h.GetXaxis().GetBinUpEdge(binx+2)
    y0, y1 = histo_p_sl1.GetBinLowEdge(1), histo_p_sl3.GetBinCenter(histo_p_sl3.GetNbinsX())

    box = r.TBox(x0, y0, x1, y1)
    box.SetFillColorAlpha(color, 0.3)
    return [histo_p_sl1, histo_p_sl2, histo_p_sl3], box, l

In [10]:
station = "MB1" # @param ["MB1", "MB2", "MB3", "MB4"]
wheel =  "1" # @param [-2, -1, 0, 1, 2]
wheel = int(wheel)

f = r.TFile.Open(thr20File)
c5 = r.TCanvas("canvas4", "", 1000, 800)
c5.Divide(2, 2)
c5.cd(1)
r.gPad.SetGridx()
h2d_7 = r.TH2I("h2d_7", "Distributions of (w_{max} - w_{min}) for digis of events with not showered genmuons",60, 0, 60, 200, 0, 200)
histos7 = [ deepcopy( f.Get( f"digi_wl3d_gmtns_MB{ist}")) for ist in range(1, 5) ]

_nd7=make_per_whstsl_3dhisto(h2d_7, histos7)
prys_7, box_7, l7 = draw_projection2(h2d_7, (station, wheel), color=r.kRed)
box_7.Draw("same")
c5.cd(3)
prys_7[0].GetXaxis().SetRangeUser(0,40)
prys_7[0].Draw()
prys_7[1].Draw("same")
prys_7[2].Draw("same")
l7.Draw("same")
r.gPad.SetFrameLineColor(r.kRed)
r.gPad.SetFrameLineWidth(4)

c5.cd(2)
h2d_8 = r.TH2I("h2d_8", "Distributions of (w_{max} - w_{min}) for digis of events with showered genmuons",60, 0, 60, 200, 0, 200)
histos8 = [ deepcopy( f.Get( f"digi_wl3d_gmts_MB{ist}")) for ist in range(1, 5) ]

_nd8=make_per_whstsl_3dhisto(h2d_8, histos8)
prys_8, box_8,l8 = draw_projection2(h2d_8, (station, wheel), color=r.kGreen)
box_8.Draw("same")
r.gPad.SetGridx()
c5.cd(4)
prys_8[0].GetXaxis().SetRangeUser(0,40)
prys_8[0].Draw()
prys_8[1].Draw("same")
prys_8[2].Draw("same")
l8.Draw("same")
r.gPad.SetFrameLineColor(r.kGreen)
r.gPad.SetFrameLineWidth(4)

c5.Draw()

Warning in <TCanvas::Constructor>: Deleting canvas with same name: canvas4


----------------------------------------------------
# **Showers plots**
see github [event_builder.py#L73](https://github.com/DanielEstrada971102/DTPatternRecognition/blob/b259f9640e06ce7122d45e248584357f4a673d94/utils/event_builder.py#L73)
* **EmuShower**: Showers built with the CMSSW emulator
* **Shower2Compare**: Showers built employing the same algorithm as EmuShowers, but implemented in Python.
* **pyShower**: Showers built in Python using half the threshold of the emuShower and analysed by SL



In [11]:
#@title emuShower variables varying threshold
def plot_shower_var(var, title, ylabel):
    emu = {"x": [], "y":[], "ex":[], "ey":[]}
    rebuilt = {"x": [], "y":[], "ex":[], "ey":[]}
    pyshower = {"x": [], "y":[], "ex":[], "ey":[]}

    for key, file in files.items():
        x = int(re.findall(r'\d+',key)[0])
        emu["x"].append(x)
        rebuilt["x"].append(x)
        pyshower["x"].append(x)
        emu["ex"].append(0)
        rebuilt["ex"].append(0)
        pyshower["ex"].append(0)

        with r.TFile.Open(file) as f:
            h_emu = f.Get("emuShowers_" + var)
            h_rebuilt = f.Get("showers2comp_" + var)
            h_pyshower = f.Get("pyShower_" + var)

            emu["y"].append(h_emu.GetMean())
            rebuilt["y"].append(h_rebuilt.GetMean())
            pyshower["y"].append(h_pyshower.GetMean())
            emu["ey"].append(h_emu.GetStdDev())
            rebuilt["ey"].append(h_rebuilt.GetStdDev())
            pyshower["ey"].append(h_pyshower.GetStdDev())

    mg = r.TMultiGraph()
    g_emushower = r.TGraphErrors(11,
        np.array(emu["x"], dtype='float'),
        np.array(emu["y"], dtype='float'),
        np.array(emu["ex"], dtype='float'),
        np.array(emu["ey"], dtype='float')
    )
    g_rebuiltshower = r.TGraphErrors(11,
        np.array(rebuilt["x"], dtype='float'),
        np.array(rebuilt["y"], dtype='float'),
        np.array(rebuilt["ex"], dtype='float'),
        np.array(rebuilt["ey"], dtype='float')
    )
    g_pyshower = r.TGraphErrors( 11,
        np.array(pyshower["x"], dtype='float'),
        np.array(pyshower["y"], dtype='float'),
        np.array(pyshower["ex"], dtype='float'),
        np.array(pyshower["ey"], dtype='float')
    )
    g_emushower.SetLineColor(r.kRed)
    g_emushower.SetLineWidth(2)
    g_emushower.SetMarkerColor(r.kRed)
    g_emushower.SetMarkerSize(1)
    g_rebuiltshower.SetLineColor(r.kBlue)
    g_rebuiltshower.SetMarkerColor(r.kBlue)
    g_pyshower.SetLineColor(r.kOrange)
    g_pyshower.SetMarkerColor(r.kOrange)

    g_emushower.SetMarkerStyle(20)
    g_rebuiltshower.SetMarkerStyle(21)
    g_pyshower.SetMarkerStyle(22)

    x0leg, y0leg, x1leg, y1leg = (0.62, 0.37, 1, 0.55)
    l1 = r.TLegend(x0leg, y0leg, x1leg, y1leg)
    l1.SetBorderSize(0)
    l1.SetFillColor(0)
    l1.SetTextFont(42)
    l1.SetTextSize(30)
    l1.SetNColumns(1)
    l1.AddEntry(g_emushower,"EmuShowers")
    l1.AddEntry(g_rebuiltshower,"showers2Compare")
    l1.AddEntry(g_pyshower,"pyShowers")

    mg.Add(g_emushower)
    mg.Add(g_rebuiltshower)
    mg.Add(g_pyshower)

    mg.SetTitle(f"{title} ; Threshold; {ylabel}")

    return mg, l1 #.Draw("AP") l1.Draw("SAME")

In [12]:
#@markdown
c3 =r.TCanvas("canvas3", "", 1000, 700)
c3.Divide(2, 2)

c3.cd(1)
mg1, l1 = plot_shower_var(var="n4ev", title="n", ylabel=" # Showers per Event #rightarrow Mean")
mg1.Draw("AP")
l1.Draw("SAME")
c3.cd(2)
mg2, l2 = plot_shower_var(var="nDigis", title="nDigis", ylabel=" # Digis in a showers per Event #rightarrow Mean")
mg2.Draw("AP")
l2.Draw("SAME")
c3.cd(3)
mg3, l3 = plot_shower_var(var="wh", title="Wheel", ylabel=" Showers wheel #rightarrow Mean")
mg3.Draw("AP")
l3.Draw("SAME")
c3.cd(4)
mg4, l4 = plot_shower_var(var="st", title="Station", ylabel=" # Showers station #rightarrow Mean")
mg4.Draw("AP")
l4.Draw("SAME")

c3.Draw()

Warning in <TCanvas::Constructor>: Deleting canvas with same name: canvas3


In [13]:
#@title N detected showers
#@markdown <font color="red">(?) I was expecting more pyShowers because the comparisons were made by SL.</font>
thrs = [12, 16, 20]
files_ = [r.TFile.Open(files[f"thr{thr}"]) for thr in thrs]

cn = r.TCanvas()

xleg = 0.6
yleg = 0.6
x0leg, y0leg, x1leg, y1leg = (xleg, yleg, xleg + .2, yleg + .2)
ln = r.TLegend(x0leg, y0leg, x1leg, y1leg)
ln.SetBorderSize(0)
ln.SetFillColor(0)
ln.SetTextFont(42)
ln.SetTextSize(10)
ln.SetNColumns(1)

for idx, file in enumerate(files_):
    nEmu_h = deepcopy( file.Get("nEmuShowers") )
    n2comp_h = deepcopy( file.Get("nShowers2Comp") )
    nPy_h = deepcopy( file.Get("nPyShowers") )

    hn = r.TH1I(f"nh{thrs[idx]}", "Number of Showers; type; n", 3, 0, 3)
    hn.SetBinContent(1, nEmu_h.GetBinContent(2))
    hn.SetBinContent(2, n2comp_h.GetBinContent(2))
    hn.SetBinContent(3, nPy_h.GetBinContent(2))
    hn.SetFillColorAlpha(idx + 1, 0.3)
    hn.SetStats(0)
    labels = ["EmuShower", "Shower2Comp", "PyShower"]
    for iBin in range(hn.GetNbinsX()):
        hn.GetXaxis().SetBinLabel(iBin + 1, labels[iBin])
    hn.Draw("SAME")
    ln.AddEntry(hn, f"Threshold {thrs[idx]}")

cn.Draw()

## Efficiencies

In [36]:
#@markdown
def make_eff_per_whst(files, histo_name, titles):
    effgrs = []
    for indx, file in enumerate(files):
        total_num = r.TH1D( f"{indx}{histo_name}_num", "", 20, 0, 20)
        total_den = r.TH1D( f"{indx}{histo_name}_den", "", 20, 0, 20)

        pre, pos = histo_name.split("MB")
        nums = [ deepcopy( file.Get( f"{pre}MB{ist}{pos[1:]}_num")) for ist in range(1, 5) ]
        dens = [ deepcopy( file.Get(f"{pre}MB{ist}{pos[1:]}_total")) for ist in range(1, 5) ]

        for iWheel in range(1, 6):
            for iStation in range(1, 5):
                iBin = 4 * (iStation-1) + iWheel + (iStation != 1)*(iStation - 1)
                total_num.SetBinContent( iBin, nums[ iStation - 1 ].GetBinContent(iWheel))
                total_den.SetBinContent( iBin, dens[ iStation - 1 ].GetBinContent(iWheel))

        eff = r.TEfficiency(total_num, total_den)
        effgr = eff.CreateGraph()

        for ibin in range(effgr.GetN()):
            effgr.SetPointEXhigh(ibin, 0)
            effgr.SetPointEXlow(ibin, 0)

        effgrs.append(effgr)

    # --- Create a frame metadata --- #
    frame = r.TH1D(f"{histo_name}_frame", titles, 20, 0, 20)
    frame.GetYaxis().SetRangeUser(-0.1, 1.05)
    frame.GetYaxis().SetTitleFont(42)
    frame.GetYaxis().SetTitleSize(0.03)
    frame.GetYaxis().SetLabelFont(42)
    frame.GetYaxis().SetLabelSize(0.022)
    frame.GetYaxis().SetTitleOffset(1.5)

    labels = ["-2", "-1", "0", "+1", "+2"]*4
    for iBin in range(frame.GetNbinsX()):
        frame.GetXaxis().SetBinLabel(iBin + 1, labels[iBin])

    frame.SetStats(0)
    frame.Draw("axis")

    for ieffgr, effgr in enumerate(effgrs):
        effgr.SetMarkerSize( 1 )
        effgr.SetLineColor( ieffgr + 1 )
        effgr.SetMarkerColor( ieffgr + 1 )
        effgr.SetMarkerStyle( 20 + ieffgr )

        effgr.Draw("PE1 SAME")

    notes = [
        ("Private work (#bf{CMS} Phase-2 Simulation)", .38, 0.925, 0.035),
        ("200 PU", .84, 0.925, 0.035),
        ("MB1", .2, .3, 0.05),
        ("MB2", .4, .3, 0.05),
        ("MB3", .6, .3, 0.05),
        ("MB4", .8, .3, 0.05),
    ]
    lines = [
        (5, -.1, 5, 1.05),
        (10, -.1, 10, 1.05),
        (15, -.1, 15, 1.05),
    ]

    _noDelete = []
    # Now add texts and lines
    for note in notes:
        text = note[0]
        x, y = note[1], note[2]
        textSize = note[3]
        latex = r.TLatex()
        latex.SetTextSize(textSize)
        latex.SetTextAlign(22)  # Centrado
        latex.SetTextFont(42)
        latex.DrawLatexNDC(x, y, text)
        _noDelete.append(latex)  # Para evitar que ROOT lo borre

    for line in lines:
            xpos0, ypos0, xpos1, ypos1 = line
            texline = deepcopy(r.TLine(xpos0, ypos0, xpos1, ypos1))
            texline.SetLineWidth(1)
            texline.Draw("same")
            _noDelete.append(texline) # So it does not get deleted by ROOT

    return frame, effgrs, _noDelete


In [37]:
#@markdown $ \rightarrow \frac{\text{showered genmuon's segments that match with any shower}}{\text{showered genmuon's segments}}$. see [baseHistos.py#L263C1-L354C5](https://github.com/DanielEstrada971102/DTPatternRecognition/blob/a3988f0875b60d1eca1ca19531b1c0e2cbe4658e/utils/baseHistos.py#L263C1-L354C5)
file_ = [r.TFile.Open(files[f"thr{12}"])]
ct = r.TCanvas("ct", "", 800, 800)
xleg = 0.12
yleg = 0.6
x0leg, y0leg, x1leg, y1leg = (xleg, yleg, xleg+0.7, yleg+0.06)

# ct.Divide(3, 1)

# ct.cd(1)
_ndeff1 = make_eff_per_whst(file_, "Seg_m_emushower_tprgm_MBX", "; Wheel;DT Local Trigger Efficiency")

l1 = r.TLegend(x0leg, y0leg, x1leg, y1leg)
l1.SetBorderSize(0)
l1.SetFillColor(0)
l1.SetTextFont(42)
l1.SetTextSize(.022)
l1.SetNColumns(1)
for idx, effgr in enumerate(_ndeff1[1]):

    l1.AddEntry(effgr,"#frac{# showered gen muon's segments matched to showers}{# showered gen muon's segments}")#+f"->Threshold {thrs[idx]}")
l1.Draw("SAME")

# ct.cd(2)
# _ndeff2 = make_eff_per_whst(files_, "Seg_m_shower2comp_tprgm_MBX", "Showers2Compare; Wheel; ")
# l2 = r.TLegend(x0leg, y0leg, x1leg, y1leg)
# l2.SetBorderSize(0)
# l2.SetFillColor(0)
# l2.SetTextFont(42)
# l2.SetTextSize(10)
# l2.SetNColumns(1)
# for idx, effgr in enumerate(_ndeff2[1]):
#     l2.AddEntry(effgr,f"Threshold {thrs[idx]}")
# l2.Draw("SAME")

# ct.cd(3)
# _ndeff3 = make_eff_per_whst(files_, "Seg_m_pyshower_tprgm_MBX", "pyShowers; Wheel;")
# l3 = r.TLegend(x0leg, y0leg, x1leg, y1leg)
# l3.SetBorderSize(0)
# l3.SetFillColor(0)
# l3.SetTextFont(42)
# l3.SetTextSize(10)
# l3.SetNColumns(1)
# for idx, effgr in enumerate(_ndeff3[1]):
#     l3.AddEntry(effgr,f"Threshold {thrs[idx]}")
# l3.Draw("SAME")

ct.SaveAs("showe_eff.svg")
ct.SaveAs("shower_eff.pdf")

ct.Draw()

Warning in <TCanvas::Constructor>: Deleting canvas with same name: ct
Info in <TCanvas::Print>: SVG file showe_eff.svg has been created
Info in <TCanvas::Print>: pdf file shower_eff.pdf has been created


In [16]:
#@markdown **Ratio of python showers that match with any emuShower**. see [baseHistos.py#L357C1-L376C5](https://github.com/DanielEstrada971102/DTPatternRecognition/blob/a3988f0875b60d1eca1ca19531b1c0e2cbe4658e/utils/baseHistos.py#L357C1-L376C5)

#@markdown <font color="red">(?) I was expecting 100% of efficiency for "showers to compare" since they should be equal than showers from CMSSW emulator.</font>
ct2 = r.TCanvas("ct2", "", 1000, 400)
xleg = 0.6
yleg = 0.6
x0leg, y0leg, x1leg, y1leg = (xleg, yleg, xleg + .2, yleg + .2)

ct2.Divide(2, 1)

ct2.cd(1)
_ndeff4 = make_eff_per_whst(files_, "eff_shower2comp_eq2Emu_MBX", "Showers to Compare; Wheel; ratio of Showeres2Comp that match with any EmuShower")

l4 = r.TLegend(x0leg, y0leg, x1leg, y1leg)
l4.SetBorderSize(0)
l4.SetFillColor(0)
l4.SetTextFont(42)
l4.SetTextSize(10)
l4.SetNColumns(1)
for idx, effgr in enumerate(_ndeff4[1]):
    l4.AddEntry(effgr,f"Threshold {thrs[idx]}")
l4.Draw("SAME")

ct2.cd(2)
_ndeff5 = make_eff_per_whst(files_, "eff_pyshower_eq2Emu_MBX", "PyShowers; Wheel;ratio of pyShowers that match with any EmuShower")
l5 = r.TLegend(x0leg, y0leg, x1leg, y1leg)
l5.SetBorderSize(0)
l5.SetFillColor(0)
l5.SetTextFont(42)
l5.SetTextSize(10)
l5.SetNColumns(1)
for idx, effgr in enumerate(_ndeff5[1]):
    l5.AddEntry(effgr,f"Threshold {thrs[idx]}")
l5.Draw("SAME")

ct2.Draw()